In [1]:
import os, subprocess
import json
import uproot3
import awkward as ak
import numpy as np
from coffea import processor, util, hist

from plotter import *

In [2]:
lumis = {}
lumis['2016'] = 35.9
lumis['2017'] = 41.5
lumis['2018'] = 59.9

with open('xsec.json') as f:
  xs = json.load(f)

with open('pmap.json') as f:
  pmap = json.load(f)

systematics = ['nominal',
               'jet_triggerUp','jet_triggerDown',
               'btagWeightUp','btagWeightDown','btagEffStatUp','btagEffStatDown',
               'UESUp','UESDown','JESUp','JESDown','JERUp','JERDown',
               'PS_weightUp','PS_weightDown'
              ]

In [3]:
year = '2017'
nfiles = len(subprocess.getoutput("ls infiles-split/"+year+"*.json").split())
outsum = processor.dict_accumulator()

In [ ]:
# Load all files
for n in range(1,nfiles+1):

    with open('infiles-split/'+year+'_'+str(n)+'.json') as f:
      infiles = json.load(f)
    
    filename = '/myeosdir/vbf-category/outfiles/'+year+'_'+str(n)+'.coffea'
    #filename = 'outfiles/'+year+'_'+str(n)+'.coffea'
    if os.path.isfile(filename):
        out = util.load(filename)
        outsum.add(out)
    else:
        print(n,infiles.keys())
        #print("File " + filename + " is missing")
        
scale_lumi = {k: xs[k] * 1000 *lumis[year] / w for k, w in outsum['sumw'].items()}
outsum['templates-vbf'].scale(scale_lumi, 'dataset')
print('ok')

In [ ]:
templates = outsum['templates-vbf'].group('dataset', hist.Cat('process', 'Process'), pmap).integrate('region','signal-vbf')
templates.sum('pt1','msd1','ddb1','deta','mjj').integrate('systematic', 'nominal').values()

In [ ]:
mc = ['QCD','Wjets','Zjets','ttbar','singlet','VV','ggF','VBF','WH','ZH']#,'ttH']

In [ ]:
h = templates.sum('pt1','ddb1').integrate('deta',int_range=slice(3.5,7)).integrate('mjj',int_range=slice(1000,4000))
g = templates.sum('pt1').integrate('deta',int_range=slice(3.5,7)).integrate('mjj',int_range=slice(1000,4000)).integrate('ddb1',int_range=slice(0.89,1))

In [ ]:
for p in mc:
    plot_syst(h.integrate('process',p),'JES',year + ' ' + p + ' JES', year+'/syst_JES/'+p)
    plot_syst(g.integrate('process',p),'JES',year + ' ' + p + ' JES DDB pass', year+'/syst_JES/'+p+"_pass")

In [ ]:
for p in mc:
    plot_syst(h.integrate('process',p),'JER',year+' '+p + ' JER',year+'/syst_JER/'+p)
    plot_syst(g.integrate('process',p),'JER',year + ' ' + p + ' JER DDB pass', year+'/syst_JER/'+p+"_pass")

In [ ]:
for p in mc:
    plot_syst(h.integrate('process',p),'btagWeight',year+' '+p + ' btagWeight',year+'/syst_btagWeight/'+p)
    plot_syst(g.integrate('process',p),'btagWeight',year + ' ' + p + ' btagWeight DDB pass', year+'/syst_btagWeight/'+p+"_pass")

In [ ]:
for p in mc:
    plot_syst(h.integrate('process',p),'btagEffStat',year+' '+p + ' btagEffStat',year+'/syst_btagEffStat/'+p)
    plot_syst(g.integrate('process',p),'btagEffStat',year + ' ' + p + ' btagEffStat DDB pass', year+'/syst_btagEffStat/'+p+"_pass")

In [ ]:
for p in mc:
    plot_syst(h.integrate('process',p),'UES',year+' '+p + ' UES',year+'/syst_UES/'+p)
    plot_syst(g.integrate('process',p),'UES',year + ' ' + p + ' UES DDB pass', year+'/syst_UES/'+p+"_pass")

In [ ]:
for p in mc:
    plot_syst(h.integrate('process',p),'jet_trigger',year+' '+p +' jet_trigger',year+'/syst_jet_trigger/'+p)
    plot_syst(g.integrate('process',p),'jet_trigger',year + ' ' + p + ' jet_trigger DDB pass', year+'/syst_jet_trigger/'+p+"_pass")

In [ ]:
for p in mc:
    plot_syst(h.integrate('process',p),'PS_weight',year+' '+p+' PS_weight',year+'/syst_PS_weight/'+p)
    plot_syst(g.integrate('process',p),'PS_weight',year + ' ' + p + ' PS_weight DDB pass', year+'/syst_PS_weight/'+p+"_pass")